In [1]:

import glob
from torchsummary import summary
import matplotlib.pyplot as plt
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import json
from PIL import Image
from IPython.display import display
import random
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist

nltk.download('punkt')
nltk.download('stopwords')
english_stopwords = set(stopwords.words('english'))
fdist = FreqDist(english_stopwords)
top_stopwords = set(word.lower() for word, _ in fdist.most_common(1000)) 
print(top_stopwords)

test_text = '''
Do you have a problem that you are struggling to solve? Why don't you ask your friends for advice? When people ask for advice on solving a problem, often times they speak to more than one person. This is because different views are better for figuring out a tough problem, many opinions are better than one, and other people may have experienced a problem like yours and may be able to help you in making better decisions.
'''

C:\Users\Nutzer\AppData\Local\Temp\ipykernel_1164\1828077501.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


{'most', 'other', 'does', 'shan', 'should', 'hers', 'both', 'can', 'itself', 'we', 'yourselves', 'couldn', "shan't", 'not', "didn't", 'on', 'mightn', 'yours', 'do', 'hasn', 'them', 'before', "that'll", 'our', 'has', 'its', 'where', 'd', 'same', 'your', 'to', "don't", "mustn't", 'about', 'is', 'nor', 'y', "isn't", 'hadn', "should've", "hasn't", 'into', "mightn't", 'ourselves', 'which', 'down', 'more', 'below', 'doing', "she's", 'won', 'shouldn', 'whom', 'such', 'again', 'my', 'these', 'further', "haven't", 'so', 'all', 'some', 'through', 'off', "hadn't", 've', 'they', 'while', 'from', 'aren', 'him', "it's", 'after', 'myself', 'been', 'the', "wasn't", 'himself', "couldn't", 'ma', 'didn', 'ours', 'out', 'their', 'those', 'yourself', "doesn't", 'but', 'ain', 'am', "you'll", 't', 'and', 'did', "weren't", 'herself', 'haven', 'his', 'were', 'only', 'theirs', 'because', 'of', 'any', 'who', 'under', 'have', "shouldn't", 'it', 'wouldn', 'each', 'm', "you're", 'me', 'own', 'themselves', 'isn', "y

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nutzer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nutzer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# AI Text Image Recognition

Is there some similarity to Malware detection, where it's common to translate malware code to bytes and then images, which are then fed into NN/DL networks? Let's test a bit.

In [2]:
ai = json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\4k\\generator_1_mistralai_Mistral-7B-v0.1_4000_outputs.json", encoding='utf8'))
#ai = ai + json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\4k\\generator_1_gpt2_4000_outputs.json", encoding='utf8'))
#ai = ai + json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\generator_1_gpt2_100_outputs.json", encoding='utf8'))

human = json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\4k\\generator_0_mistralai_Mistral-7B-v0.1_4000_outputs.json", encoding='utf8'))
#human = human + json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\4k\\generator_0_gpt2_4000_outputs.json", encoding='utf8'))
#human = human + json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\generator_0_gpt2_100_outputs.json", encoding='utf8'))
print(len(ai))
print(len(human))

3992
3999


## Vis as Image


In [3]:
def output_to_image(detector_outputs):
    for output in detector_outputs:
        meta = output['metadata']
        height = meta['sample_rate']
        width = meta['sample_sequence_length']
        results = output['ensemble_results'][0]['sample_results']
        
        image = Image.new('RGBA', (100, 20), color=(0,0,0,255))
        pixels = image.load()

        x = 1
        offset_x = x
        y = 1

        for result in results:
            steps = result['model_outputs']['steps']
            steps.sort(key= lambda x: x['step'])

            for step in steps:                
                r = 255
                g = 255
                b = 255
                
                # Let's try to ignore the stopwords as they are probably useless
                vis = int(step['target_prob'] * 255)
                if step['target'].lower() in english_stopwords:
                    vis = 0

                all_probs = step['top_k_probs']
                all_probs.sort(reverse=True)
                other_probs = all_probs[1:]
                
                # mid
                pixels[x, y] = (r, g, b, vis)
                # right
                pixels[x+1, y] = (r, g, b, int(other_probs[0] * 255))
                # left
                pixels[x-1, y] = (r, g, b, int(other_probs[1] * 255))
                # down
                pixels[x, y-1] = (r, g, b, int(other_probs[2] * 255))
                # up
                pixels[x, y+1] = (r, g, b, int(other_probs[3] * 255))
                
                x += 3
            
            offset_x = int((offset_x % 2 == 1))
            x = offset_x
            y += 3
        display(image)

def draw_image(width, height):
    image = Image.new('RGBA', (width, height))
    pixels = image.load()
    # iterate over all pixels
    for y in range(height):
        for x in range(width):
            r = random.randint(0, 255)
            g = random.randint(0, 255)
            b = random.randint(0, 255)
            a = random.randint(0, 1)
            pixels[x, y] = (r, g, b, int(a * 255))  # Set the pixel RGBA values
    return image
    

In [4]:
output_to_image(human[:2])
output_to_image(ai[:2])

# CNN

Let's try some CNN detection on the steps

## 2D CNN

Let's start by creating 2d arrays with x channels per "pixel"

In [5]:

def create_dataset(detector_outputs, label):
    data = []
    labels = []
    
    for output in detector_outputs:
        results = output['ensemble_results'][0]['sample_results']
        rows = []

        for result in results:
            steps = result['model_outputs']['steps']
            steps.sort(key= lambda x: x['step'])
            columns = []
            for step in steps:
                prob = step['target_prob']
                # Common target stepwords, we mask out
                if step['target'].lower() in english_stopwords:
                    prob = 0
                channels = [prob]
                # Again, mask out common stopwords
                top_k_tokens = step['top_k_tokens']
                top_k_probs = step['top_k_probs']
                for i in range(0 , len(top_k_tokens)):
                    token = top_k_tokens[i]
                    if token.lower() in english_stopwords:
                        top_k_probs[i] = 0
                top_k_probs.sort()
                channels = channels + top_k_probs
                columns.append(channels)
                
            rows.append(columns)
          
        data.append(rows)
        labels.append(label)  
    data = np.array(data)
    labels = np.array(labels)
    
    return data, labels

In [6]:
data_ai, labels_ai = create_dataset(ai, 1)
data_hu, labels_hu = create_dataset(human, 0)
data = np.concatenate((data_ai, data_hu), axis=0)
labels = np.concatenate((labels_hu, labels_ai))
print(len(labels))

# Convert numpy arrays to tensors
# We need: [batch_size, channels, height, width]
# data_tensor = torch.tensor(data, dtype=torch.float32).permute(0, 2, 1).unsqueeze(-2)
data_tensor = torch.tensor(data, dtype=torch.float32).permute(0, 3, 1, 2)
print(data_tensor.shape)
labels_tensor = torch.tensor(labels, dtype=torch.long)

print(data_tensor[:1])
print(labels_tensor[:1])

7991
torch.Size([7991, 11, 6, 32])
tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           8.7171e-01, 0.0000e+00],
          [0.0000e+00, 2.0752e-02, 3.5485e-01,  ..., 0.0000e+00,
           7.4845e-01, 0.0000e+00],
          [3.8078e-02, 5.5823e-01, 7.6015e-01,  ..., 0.0000e+00,
           0.0000e+00, 8.7745e-01],
          [3.8229e-01, 3.2795e-01, 9.9991e-01,  ..., 0.0000e+00,
           2.4257e-01, 9.4451e-01],
          [3.5837e-02, 0.0000e+00, 8.9047e-01,  ..., 0.0000e+00,
           8.6310e-01, 9.9965e-01],
          [3.3034e-01, 0.0000e+00, 5.0898e-01,  ..., 0.0000e+00,
           6.3373e-01, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 1.1958e-03, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 1.0631e-05, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 1.0815e-07],
          [0.0000e+00, 0.0000e+00, 5.1296e-11,  ..., 0

# Train, Test Split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data_tensor, labels_tensor, test_size=0.2, random_state=42)
print("x_train shape",X_train.shape)
print("x_test shape",X_test.shape)
print("y_train shape",y_train.shape)
print("y_test shape",y_test.shape)
print(y_test[:20])

x_train shape torch.Size([6392, 11, 6, 32])
x_test shape torch.Size([1599, 11, 6, 32])
y_train shape torch.Size([6392])
y_test shape torch.Size([1599])
tensor([1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0])


# Model


In [8]:
model = nn.Sequential(
    nn.Conv2d(in_channels=11, out_channels=32, kernel_size=(2,8), padding=(0, 1)),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(1,2), stride=(1, 2)),
    nn.Dropout(p=0.25),

    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(2,6), padding=(0,1)),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(1,2), stride=(1,2)),
    nn.Dropout(p=0.25),

    nn.Flatten(),
    nn.Linear(in_features=20, out_features=64),
    nn.ReLU(),
    nn.Dropout(p=0.4),

    nn.Linear(in_features=64, out_features=1),
    nn.Sigmoid()
)

print(model)

Sequential(
  (0): Conv2d(11, 32, kernel_size=(2, 8), stride=(1, 1), padding=(0, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (3): Dropout(p=0.25, inplace=False)
  (4): Conv2d(32, 64, kernel_size=(2, 6), stride=(1, 1), padding=(0, 1))
  (5): ReLU()
  (6): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (7): Dropout(p=0.25, inplace=False)
  (8): Flatten(start_dim=1, end_dim=-1)
  (9): Linear(in_features=20, out_features=64, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.4, inplace=False)
  (12): Linear(in_features=64, out_features=1, bias=True)
  (13): Sigmoid()
)


In [9]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

e:\Python_Projects\PrismAI\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Training the model
num_epochs = 200
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    model.train()  # Set the model to training mode
    for inputs, labels in zip(X_train, y_train):
        optimizer.zero_grad()
        outputs = model(inputs)
        
        if len(labels.size()) == 0:
            labels = labels.unsqueeze(0).expand(64, 1).float()
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        # Calculate training accuracy
        predicted = outputs > 0.5  # Assuming threshold of 0.5 for binary classification
        total += labels.size(0)
        correct += (predicted == labels.byte()).sum().item()
    
    # Print training loss and accuracy
    train_loss = running_loss / len(y_train)
    train_accuracy = correct / total
    print(f"Epoch {epoch+1}, Training Loss: {train_loss:0.5f}, Training Accuracy: {train_accuracy:0.5f}")
    
    # Test loss and accuracy calculation
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for test_inputs, test_labels in zip(X_test, y_test):
            test_outputs = model(test_inputs)
            if len(test_labels.size()) == 0:
                test_labels = test_labels.unsqueeze(0).expand(64, 1).float()
            test_loss += criterion(test_outputs, test_labels).item()
            test_predicted = test_outputs > 0.5
            test_total += test_labels.size(0)
            test_correct += (test_predicted == test_labels.byte()).sum().item()
    
    # Print test loss and accuracy
    test_loss /= len(y_test)
    test_accuracy = test_correct / test_total
    print(f"Epoch {epoch+1}, Test Loss: {test_loss:0.5f}, Test Accuracy: {test_accuracy:0.5f}\n")


Epoch 1, Training Loss: 0.56728, Training Accuracy: 0.69439
Epoch 1, Test Loss: 0.49594, Test Accuracy: 0.75382

Epoch 2, Training Loss: 0.50459, Training Accuracy: 0.75479
Epoch 2, Test Loss: 0.46880, Test Accuracy: 0.77011

Epoch 3, Training Loss: 0.48457, Training Accuracy: 0.76568
Epoch 3, Test Loss: 0.45256, Test Accuracy: 0.77352

Epoch 4, Training Loss: 0.47039, Training Accuracy: 0.76791
Epoch 4, Test Loss: 0.43976, Test Accuracy: 0.77520

Epoch 5, Training Loss: 0.45740, Training Accuracy: 0.76912
Epoch 5, Test Loss: 0.43318, Test Accuracy: 0.77429

Epoch 6, Training Loss: 0.44368, Training Accuracy: 0.77165
Epoch 6, Test Loss: 0.41186, Test Accuracy: 0.77558

Epoch 7, Training Loss: 0.42075, Training Accuracy: 0.77282
Epoch 7, Test Loss: 0.38879, Test Accuracy: 0.77445

Epoch 8, Training Loss: 0.40703, Training Accuracy: 0.77761
Epoch 8, Test Loss: 0.38036, Test Accuracy: 0.77593

Epoch 9, Training Loss: 0.39857, Training Accuracy: 0.78390
Epoch 9, Test Loss: 0.36899, Test Ac